<a href="https://colab.research.google.com/github/pungji01/dataAnalysis/blob/main/3_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%B6%84%EB%A5%98_1_%EC%82%AC%EC%9D%B4%ED%82%B7%EB%9F%B0_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%EC%83%9D%EC%A1%B4%EC%9E%90%EC%98%88%EC%B8%A1_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 타이타닉 생존자 예측 : 분류

## #사용 모델 : RandomForestClassifier

## 방법1: 사이킷런을 이용한 기본 방법

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# 데이터 불러오기
train_data_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_data_url)

# 불필요한 컬럼 제거
data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# 결측치 처리
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# 범주형 변수 인코딩 (One-Hot Encoding)
encoder = OneHotEncoder(sparse=False)
categorical_columns = ['Sex', 'Embarked', 'Pclass']
encoded_categorical = encoder.fit_transform(data[categorical_columns])

# 수치형 변수 표준화
scaler = StandardScaler()
numerical_columns = ['Age', 'SibSp', 'Parch', 'Fare']
scaled_numerical = scaler.fit_transform(data[numerical_columns])

# 인코딩된 범주형 데이터 및 표준화된 수치형 데이터를 결합
import numpy as np
X = np.hstack([scaled_numerical, encoded_categorical])

# 타겟 데이터 분리
y = data['Survived']

# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성 및 학습
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 예측 수행
y_pred = clf.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# 자세한 성능 평가 보고서 출력
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Test Accuracy: 82.12%
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       105
           1       0.79      0.77      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



## 방법2: **컬럼 변환기**를 사용한 전처리(ColumnTransformer)
방법1을 조금 더 간단하게 하는 방법

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# 데이터 불러오기
train_data_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_data_url)

# 불필요한 컬럼 제거
data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# 결측치 처리
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


# 범주형 변수와 수치형 변수를 구분
categorical_features = ['Sex', 'Embarked', 'Pclass']
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare']

# 컬럼 변환기를 통한 전처리
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# 피처와 타겟 분리
X = data.drop('Survived', axis=1)
y = data['Survived']

# 훈련 및 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 랜덤 포레스트 모델 생성 및 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 모델 학습
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# 자세한 성능 평가 보고서 출력
print('성능 평가 보고서: \n', classification_report(y_test, y_pred))

Test Accuracy: 82.12%
성능 평가 보고서: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       105
           1       0.79      0.77      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



## 방법3: **파이프라인**을 이용하여 전처리와 여러 개의 모델 적용 한번에(Pipeline)
여러개의 모델을 동시에 학습시킬 수 있다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# 데이터 불러오기
train_data_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_data_url)

# 불필요한 컬럼 제거
data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# 결측치 처리
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


# 범주형 변수와 수치형 변수를 구분
categorical_features = ['Sex', 'Embarked', 'Pclass']
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare']

# 컬럼 변환기를 통한 전처리
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# 피처와 타겟 분리
X = data.drop('Survived', axis=1)
y = data['Survived']

# 훈련 및 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 파이프라인 구축 (전처리 + 분류 모델)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))     # 이부분에 여러 개의 다른 모델을 추가할 수 있다
])

# 모델 학습
pipeline.fit(X_train, y_train)

# 예측 수행
y_pred = pipeline.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# 자세한 성능 평가 보고서 출력
print('성능 평가 보고서: \n', classification_report(y_test, y_pred))

Test Accuracy: 82.12%
성능 평가 보고서: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       105
           1       0.79      0.77      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

